In [1]:
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from PIL import Image

import glob
import os
import re

cwd = os.getcwd()

%matplotlib inline

In [3]:
SAMPLE_SIZE = 6270
SPLIT_RATIO = 0.7

IMAGE_SIZE = 256
BATCH_SIZE = 16

In [4]:
index = np.arange(SAMPLE_SIZE) + 1
np.random.seed(1234)
np.random.shuffle(index)

split = (int) (SAMPLE_SIZE * SPLIT_RATIO)
x_train_idx = index[0:split]
x_test_idx = index[split:]

In [2]:
def read_image(image_file):
    image = Image.open(image_file).convert('L')
    image = image.convert()
    image = image.resize((IMAGE_SIZE, IMAGE_SIZE))
    image = np.asarray(image).reshape((IMAGE_SIZE, IMAGE_SIZE, 1))
    return image / 255

In [ ]:
def input_generator(indexs):
    files = glob.glob("./data/*")
    
    while(True):
        x_image = []
        y_image = []
        
        for idx in indexs:
            picture_files = list(filter(re.compile(".\/data\/sample-" + str(idx) + "-\d+.jpg").search, files))
            
            for picture in picture_files:
                image = read_image(picture)
                picture = picture.split(".")
                image_blank = read_image(picture[0] + "-blank." + picture[1])
                
                x_image.append(image)
                y_image.append(image_blank)

                if len(y) >= BATCH_SIZE:
                    yield np.array(x_image), np.array(y)
                    x_image = []
                    y_image = []

In [20]:
from keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, Reshape, Conv2DTranspose, UpSampling2D
from keras.optimizers import RMSprop
from keras.models import Model

def generate_model():
    image_input = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 1))
    
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    img = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
    img = MaxPooling2D((4,4))(img)
    img = Conv2D(64, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(64, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D((4,4))(img)
    img = Dropout(0.25)(img)
    img = Conv2D(128, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(128, (3, 3), padding='same', activation='relu')(img)
    img = MaxPooling2D((2,2))(img)
    img = Dropout(0.25)(img)
    img = Conv2D(256, (3, 3), padding='same', activation='relu')(img)
    img = Conv2D(256, (3, 3), padding='same', activation='relu')(img)
    
    img = Flatten()(img)
    img = Dense(1024, activation='relu')(img)
    img = Dropout(0.3)(img)
    img = Dense(1024, activation='relu')(img)
    img = Dropout(0.3)(img)
    
    img = Dense(1024, activation='relu')(img)
    img = Dropout(0.3)(img)
    
    img = Dense(16384, activation='relu')(img)
    img = Dropout(0.3)(img)
    img = Reshape((8, 8, 256))(img)
    
    img = Conv2DTranspose(256, (3, 3), padding='same', activation='relu')(img)
    img = Dropout(0.25)(img)
    img = UpSampling2D((2,2))(img)
    img = Conv2DTranspose(128, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(128, (3, 3), padding='same', activation='relu')(img)
    img = Dropout(0.25)(img)
    img = UpSampling2D((4,4))(img)
    img = Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(64, (3, 3), padding='same', activation='relu')(img)
    img = UpSampling2D((4,4))(img)
    img = Conv2DTranspose(32, (3, 3), padding='same', activation='relu')(img)
    img = Conv2DTranspose(1, (3, 3), padding='same', activation='sigmoid')(img)

    outputs = img

    optimizer = RMSprop()
    model = Model(inputs=image_input, outputs=outputs)
    model.compile(optimizer=optimizer,
                  loss='mse')
    
    return model

print(generate_model().summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 256, 256, 1)       0         
_________________________________________________________________
conv2d_96 (Conv2D)           (None, 256, 256, 32)      320       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_97 (Conv2D)           (None, 64, 64, 64)        18496     
_________________________________________________________________
conv2d_98 (Conv2D)           (None, 64, 64, 64)        36928     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_89 (Dropout)         (None, 16, 16, 64)        0         
__________

In [ ]:
from keras.callbacks import Callback
from keras import backend as K

class TensorBoard(Callback):

    def __init__(self, log_dir='./logs',
                 write_graph=False,
                 start_steps=0,
                 batch_freq=1):
        super(TensorBoard, self).__init__()
        
        global tf, projector
        import tensorflow as tf
        from tensorflow.contrib.tensorboard.plugins import projector
        
        self.log_dir = log_dir
        self.batch_freq = batch_freq
        self.write_graph = write_graph
        
        self.start_steps = start_steps
        self.steps_counter = 1

    def set_model(self, model):
        self.model = model
        self.sess = K.get_session()
        
        self.merged = tf.summary.merge_all()

        if self.write_graph:
            self.writer = tf.summary.FileWriter(self.log_dir,
                                                self.sess.graph)
        else:
            self.writer = tf.summary.FileWriter(self.log_dir)
            
    def save_scalar(self, logs):
        log = logs or {}
        
        for name, value in logs.items():
            if name in ['batch', 'size']:
                continue
            summary = tf.Summary()
            summary_value = summary.value.add()
            summary_value.simple_value = value.item()
            summary_value.tag = name
            self.writer.add_summary(
                summary,
                self.start_steps + self.steps_counter
            )
        self.writer.flush()
            
    def on_batch_end(self, batch, logs=None):
        if self.steps_counter % self.batch_freq == 0:
            self.save_scalar(logs)
        self.steps_counter += 1

    def on_epoch_end(self, epoch, logs=None):
        self.save_scalar(logs)        

    def on_train_end(self, _):
        self.writer.close()

class ModelCheckpoint(Callback):

    def __init__(self,
                 filepath,
                 start_steps=0,
                 batch_freq=1):
        super(ModelCheckpoint, self).__init__()
        
        self.filepath = filepath
        
        self.steps_counter = 0
        self.start_steps = start_steps
        self.batch_freq = batch_freq
        
        self.steps_counter
        
    def save_model(self):
        self.model.save_weights(self.filepath, overwrite=True)
        
    def on_batch_end(self, batch, logs=None):
        if self.steps_counter % self.batch_freq == 0:
            self.save_model()
        self.steps_counter += 1
        
    def on_epoch_end(self, epoch, logs=None):
        self.save_model()

In [ ]:
model = generate_model()
model.load_weights('./model/model-weight.hdf5')

model.fit_generator(
    input_generator(x_train_idx),
    steps_per_epoch=30443,
    validation_data=input_generator(x_test_idx),
    validation_steps=3382,
    max_queue_size=5,
    epochs=3,
    use_multiprocessing=True,
    callbacks=[
        TensorBoard(
            log_dir="./model/logs/",
            batch_freq=20
        ),
        ModelCheckpoint(
            filepath="./model/model-weight.hdf5",
            batch_freq=100
        )
    ]
)